In [55]:
import pandas as pd
import numpy as np
#coding=utf-8
import numpy as np
import pandas as pd
from scipy.stats import mode
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from matplotlib import  pyplot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn import preprocessing  
min_max_scaler = preprocessing.MinMaxScaler()  
from sklearn.model_selection import KFold,StratifiedKFold

In [69]:
train = pd.read_csv("../input/train_recieve_addr_info.csv")
test = pd.read_csv("../input/test_recieve_addr_info.csv")
fulldata = pd.concat([train,test],axis = 0,ignore_index = True)
train.head(),train.shape,test.shape,fulldata.shape

(       addr_id         region        phone    fix_phone                  id  \
 0          NaN            NaN          NaN          NaN  575150029610684424   
 1  137816020.0   辽宁营口市鲅鱼圈区芦屯镇  186****4600  186****4600  563888070781309192   
 2  137630418.0  辽宁营口市鲅鱼圈区红海街道  159****1000  159****1000  563888070781309192   
 3  137816021.0   辽宁营口市鲅鱼圈区芦屯镇  186****4600          NaN  563888070781309192   
 4  138135925.0   辽宁营口市鲅鱼圈区芦屯镇  189****9687  189****9687  563888070781309192   
 
                        receiver_md5  
 0                               NaN  
 1  798371eaefb014d304e8c90ba0af1d09  
 2  41c14ef94733b4587f11ffb64e58a72d  
 3  798371eaefb014d304e8c90ba0af1d09  
 4  2d3e7dc595265a37f1e9eee82a15b047  , (261299, 6), (80382, 6), (341681, 6))

In [72]:
fulldata = pd.concat([train,test],axis = 0,ignore_index = True)
mm = fulldata.region.value_counts().sort_values(ascending = False).index[0]
fulldata.region.fillna(mm,inplace = True)
print(fulldata.isnull().sum())
fulldata = fulldata[["id","region"]]
print(fulldata.shape,len(fulldata.id.unique()))
fulldata

addr_id         137107
region               0
phone            49940
fix_phone       179681
id                   0
receiver_md5     49936
dtype: int64
(341681, 2) 167960


,id,region
0,575150029610684424,广东深圳市宝安区
1,563888070781309192,辽宁营口市鲅鱼圈区芦屯镇
2,563888070781309192,辽宁营口市鲅鱼圈区红海街道
3,563888070781309192,辽宁营口市鲅鱼圈区芦屯镇
4,563888070781309192,辽宁营口市鲅鱼圈区芦屯镇
5,563888070781309192,辽宁营口市鲅鱼圈区芦屯镇
6,563888070781309192,辽宁营口市鲅鱼圈区海星街道
7,607668436595642632,广东深圳市宝安区
8,20160508130002849573,四川广元市利州区城区
9,20160508130002849573,四川广元市利州区东坝街道


In [73]:
#地址信息统计
#截取两个中文字符
def start_2word1(string):
    string = str(string)
    string = string[0:2]
    return string
def start_2word2(string):
    string = str(string)
    string = string[0:5]
    return string
def start_2word3(string):
    string = str(string)
    string = string[-1]
    return string
fulldata["region_pro"] = fulldata["region"].apply(lambda x: start_2word1(x))
fulldata["region_city"] = fulldata["region"].apply(lambda x: start_2word2(x))
fulldata["region_diji"] = fulldata["region"].apply(lambda x: start_2word3(x))
print(fulldata.shape,fulldata.isnull().sum())
fulldata

(341681, 5) id             0
region         0
region_pro     0
region_city    0
region_diji    0
dtype: int64


,id,region,region_pro,region_city,region_diji
0,575150029610684424,广东深圳市宝安区,广东,广东深圳市,区
1,563888070781309192,辽宁营口市鲅鱼圈区芦屯镇,辽宁,辽宁营口市,镇
2,563888070781309192,辽宁营口市鲅鱼圈区红海街道,辽宁,辽宁营口市,道
3,563888070781309192,辽宁营口市鲅鱼圈区芦屯镇,辽宁,辽宁营口市,镇
4,563888070781309192,辽宁营口市鲅鱼圈区芦屯镇,辽宁,辽宁营口市,镇
5,563888070781309192,辽宁营口市鲅鱼圈区芦屯镇,辽宁,辽宁营口市,镇
6,563888070781309192,辽宁营口市鲅鱼圈区海星街道,辽宁,辽宁营口市,道
7,607668436595642632,广东深圳市宝安区,广东,广东深圳市,区
8,20160508130002849573,四川广元市利州区城区,四川,四川广元市,区
9,20160508130002849573,四川广元市利州区东坝街道,四川,四川广元市,道


In [74]:
def transf(x):
    x = str(x)
    if x.startswith(('镇','乡')):
        x= 1
    else:
        x = 2
    return x
fulldata.region_diji = fulldata.region_diji.apply(lambda x: transf(x))
fulldata.region_diji.value_counts()

2    236624
1    105057
Name: region_diji, dtype: int64

In [75]:
def mysplit6(x):
    return x.split("(")[0] 
fulldata.region = fulldata.region.apply(lambda x: mysplit6(x))
def mysplit7(x):
    return x.split("（")[0]  
fulldata.region = fulldata.region.apply(lambda x: mysplit7(x))
fulldata11 = fulldata.copy()

print(fulldata.shape,'\n',fulldata.isnull().sum())
fulldata

(341681, 5) 
 id             0
region         0
region_pro     0
region_city    0
region_diji    0
dtype: int64


,id,region,region_pro,region_city,region_diji
0,575150029610684424,广东深圳市宝安区,广东,广东深圳市,2
1,563888070781309192,辽宁营口市鲅鱼圈区芦屯镇,辽宁,辽宁营口市,1
2,563888070781309192,辽宁营口市鲅鱼圈区红海街道,辽宁,辽宁营口市,2
3,563888070781309192,辽宁营口市鲅鱼圈区芦屯镇,辽宁,辽宁营口市,1
4,563888070781309192,辽宁营口市鲅鱼圈区芦屯镇,辽宁,辽宁营口市,1
5,563888070781309192,辽宁营口市鲅鱼圈区芦屯镇,辽宁,辽宁营口市,1
6,563888070781309192,辽宁营口市鲅鱼圈区海星街道,辽宁,辽宁营口市,2
7,607668436595642632,广东深圳市宝安区,广东,广东深圳市,2
8,20160508130002849573,四川广元市利州区城区,四川,四川广元市,2
9,20160508130002849573,四川广元市利州区东坝街道,四川,四川广元市,2


In [76]:
lat3 = pd.read_csv("../input/kkdeal_geo_ok.csv")     
Latitude = {}  #纬线
longitude = {}  #经度，且经度大
for i in range(lat3.shape[0]):
    Latitude[lat3.Address.iloc[i]] =  lat3.LATB.iloc[i]
    longitude[lat3.Address.iloc[i]] =  lat3.LNGB.iloc[i]
lat5 = pd.read_csv("../input/kk2_geo_ok.csv")
for i in range(lat5.shape[0]):
    Latitude[lat5.Address.iloc[i]] =  lat5.LATB.iloc[i]
    longitude[lat5.Address.iloc[i]] =  lat5.LNGB.iloc[i]
Latitude,longitude

({'辽宁营口市鲅鱼圈区红海街道': 40.248937,
  '辽宁营口市鲅鱼圈区海星街道': 40.294837,
  '四川广元市利州区东坝街道': 32.436609999999995,
  '四川遂宁市蓬溪县县城内': 30.657490999999997,
  '江苏淮安市涟水县南集镇': 33.764573,
  '安徽六安市霍邱县河口镇': 32.141339,
  '河北唐山市乐亭县城区': 39.357229,
  '广西崇左市大新县龙门乡': 22.889127,
  '湖南邵阳市新宁县黄龙镇': 26.540533,
  '广西梧州市苍梧县沙头镇': 23.944079000000002,
  '江苏常州市金坛区朱林镇': 31.725281,
  '甘肃白银市平川区水泉乡': 36.901571999999994,
  '陕西咸阳市乾县新阳镇': 34.500976,
  '广西梧州市藤县金鸡镇': 23.232557999999997,
  '浙江杭州市余杭区临平街道': 30.448443,
  '江西九江市武宁县宋溪镇': 29.30785,
  '江西宜春市上高县锦江镇': 28.262545000000003,
  '河南新乡市红旗区小店镇': 35.313784999999996,
  '河南漯河市源汇区大刘镇': 33.527366,
  '广东清远市佛冈县龙山镇': 23.803615,
  '浙江宁波市象山县爵溪街道': 29.482938,
  '广东汕头市南澳县云澳镇': 23.413879,
  '河南驻马店市新蔡县佛阁寺镇': 32.69829,
  '河南南阳市卧龙区百里奚街道': 33.003743,
  '河南漯河市郾城区新店镇': 33.656702,
  '云南大理州巍山县大仓镇': 25.386131,
  '吉林长春市二道区(北环城路以南)': 43.958451000000004,
  '湖北襄阳市襄州区古驿镇': 32.294056,
  '安徽铜陵市枞阳县白湖乡': 30.974721999999996,
  '福建三明市沙县现代物流产业开发区': 26.446506,
  '甘肃平凉市静宁县城区': 35.434012,
  '江西吉安市井冈山市古城镇': 26.761241,
  '北京顺义

In [77]:
lat1 = pd.read_excel("../input/train_region.xlsx") 
lat2 = pd.read_excel("../input/test_region.xlsx") 

lat = pd.concat([lat2,lat1],axis = 0,ignore_index = True)
print(lat.shape)
# lat = lat.
lat.drop_duplicates(['parse'],inplace = True)
print(lat.shape)
lat

(62197, 1)
(36217, 1)


,parse
0,"福建福州市连江县马鼻镇:99.921606,24.59032"
1,"云南临沧市凤庆县凤山镇:99.921606,24.59032"
2,"广东深圳市宝安区:113.930013,22.707433"
4,"广东东莞市常平镇:114.024589,22.979369"
5,"山东潍坊市潍城区:113.763434,23.043024"
6,"云南迪庆州维西县:99.152723,27.452958"
7,"湖南长沙市岳麓区西湖街道:112.94425,28.209835"
8,"云南昆明市官渡区城区:102.828819,25.031311"
9,"福建厦门市湖里区:118.144676,24.521974"
11,"内蒙古呼伦贝尔市鄂伦春旗甘河镇:106.724173,26.541413"


In [78]:
def mysplit1(x):
    return x.split(":")[0]
def mysplit2(x):
    return x.split(":")[1]
def mysplit3(x):
    return x.split(",")[1]
def mysplit4(x):
    return x.split(",")[0]
lat["cityName"] = lat.parse.apply(lambda x: mysplit1(x))
lat["cityLl"] = lat.parse.apply(lambda x: mysplit2(x))

lat["Latitude"] = lat.cityLl.apply(lambda x: mysplit3(x))
lat["longitude"] = lat.cityLl.apply(lambda x: mysplit4(x))
lat

,parse,cityName,cityLl,Latitude,longitude
0,"福建福州市连江县马鼻镇:99.921606,24.59032",福建福州市连江县马鼻镇,"99.921606,24.59032",24.59032,99.921606
1,"云南临沧市凤庆县凤山镇:99.921606,24.59032",云南临沧市凤庆县凤山镇,"99.921606,24.59032",24.59032,99.921606
2,"广东深圳市宝安区:113.930013,22.707433",广东深圳市宝安区,"113.930013,22.707433",22.707433,113.930013
4,"广东东莞市常平镇:114.024589,22.979369",广东东莞市常平镇,"114.024589,22.979369",22.979369,114.024589
5,"山东潍坊市潍城区:113.763434,23.043024",山东潍坊市潍城区,"113.763434,23.043024",23.043024,113.763434
6,"云南迪庆州维西县:99.152723,27.452958",云南迪庆州维西县,"99.152723,27.452958",27.452958,99.152723
7,"湖南长沙市岳麓区西湖街道:112.94425,28.209835",湖南长沙市岳麓区西湖街道,"112.94425,28.209835",28.209835,112.94425
8,"云南昆明市官渡区城区:102.828819,25.031311",云南昆明市官渡区城区,"102.828819,25.031311",25.031311,102.828819
9,"福建厦门市湖里区:118.144676,24.521974",福建厦门市湖里区,"118.144676,24.521974",24.521974,118.144676
11,"内蒙古呼伦贝尔市鄂伦春旗甘河镇:106.724173,26.541413",内蒙古呼伦贝尔市鄂伦春旗甘河镇,"106.724173,26.541413",26.541413,106.724173


In [79]:
for i in range(lat.shape[0]):
    Latitude[lat.cityName.iloc[i]] =  lat.Latitude.iloc[i]
    longitude[lat.cityName.iloc[i]] =  lat.longitude.iloc[i]
Latitude,longitude

({'辽宁营口市鲅鱼圈区红海街道': 40.248937,
  '辽宁营口市鲅鱼圈区海星街道': 40.294837,
  '四川广元市利州区东坝街道': 32.436609999999995,
  '四川遂宁市蓬溪县县城内': 30.657490999999997,
  '江苏淮安市涟水县南集镇': 33.764573,
  '安徽六安市霍邱县河口镇': 32.141339,
  '河北唐山市乐亭县城区': 39.357229,
  '广西崇左市大新县龙门乡': 22.889127,
  '湖南邵阳市新宁县黄龙镇': 26.540533,
  '广西梧州市苍梧县沙头镇': 23.944079000000002,
  '江苏常州市金坛区朱林镇': 31.725281,
  '甘肃白银市平川区水泉乡': 36.901571999999994,
  '陕西咸阳市乾县新阳镇': 34.500976,
  '广西梧州市藤县金鸡镇': 23.232557999999997,
  '浙江杭州市余杭区临平街道': 30.448443,
  '江西九江市武宁县宋溪镇': 29.30785,
  '江西宜春市上高县锦江镇': 28.262545000000003,
  '河南新乡市红旗区小店镇': 35.313784999999996,
  '河南漯河市源汇区大刘镇': 33.527366,
  '广东清远市佛冈县龙山镇': 23.803615,
  '浙江宁波市象山县爵溪街道': 29.482938,
  '广东汕头市南澳县云澳镇': 23.413879,
  '河南驻马店市新蔡县佛阁寺镇': 32.69829,
  '河南南阳市卧龙区百里奚街道': 33.003743,
  '河南漯河市郾城区新店镇': 33.656702,
  '云南大理州巍山县大仓镇': 25.386131,
  '吉林长春市二道区(北环城路以南)': 43.958451000000004,
  '湖北襄阳市襄州区古驿镇': 32.294056,
  '安徽铜陵市枞阳县白湖乡': 30.974721999999996,
  '福建三明市沙县现代物流产业开发区': 26.446506,
  '甘肃平凉市静宁县城区': 35.434012,
  '江西吉安市井冈山市古城镇': 26.761241,
  '北京顺义

In [85]:
def findll1(x):
    global Latitude    
    if x not in Latitude:
        return np.nan
    return Latitude[x]
def findll2(x):
    global longitude   
    if x not in longitude:
        return np.nan
    return longitude[x]

fulldata["Latitude"] = fulldata.region.apply(lambda x:findll1(x))
fulldata["longitude"] = fulldata.region.apply(lambda x:findll2(x))
fulldata.isnull().sum()
# zh = fulldata.loc[fulldata.Latitude.isnull()]
# zh = zh[["id","region"]]
# # zh.to_csv("../input/kk2.csv",encoding = 'gbk',index = False)
# zh

id             0
region         0
region_pro     0
region_city    0
region_diji    0
Latitude       0
longitude      0
dtype: int64

In [86]:
fulldata.to_csv("fulldata_region_unCount.csv",index = False)
fulldata

,id,region,region_pro,region_city,region_diji,Latitude,longitude
0,575150029610684424,广东深圳市宝安区,广东,广东深圳市,2,32.212549,119.92271
1,563888070781309192,辽宁营口市鲅鱼圈区芦屯镇,辽宁,辽宁营口市,1,40.285469,122.262267
2,563888070781309192,辽宁营口市鲅鱼圈区红海街道,辽宁,辽宁营口市,2,40.2489,122.139
3,563888070781309192,辽宁营口市鲅鱼圈区芦屯镇,辽宁,辽宁营口市,1,40.285469,122.262267
4,563888070781309192,辽宁营口市鲅鱼圈区芦屯镇,辽宁,辽宁营口市,1,40.285469,122.262267
5,563888070781309192,辽宁营口市鲅鱼圈区芦屯镇,辽宁,辽宁营口市,1,40.285469,122.262267
6,563888070781309192,辽宁营口市鲅鱼圈区海星街道,辽宁,辽宁营口市,2,40.2948,122.141
7,607668436595642632,广东深圳市宝安区,广东,广东深圳市,2,32.212549,119.92271
8,20160508130002849573,四川广元市利州区城区,四川,四川广元市,2,22.609761,114.150975
9,20160508130002849573,四川广元市利州区东坝街道,四川,四川广元市,2,32.4366,105.865


In [ ]:
fulldata = pd.read_csv("fulldata_region_unCount.csv")

In [94]:
print(fulldata.shape)
fulldata.head()

(341681, 7)


,id,region,region_pro,region_city,region_diji,Latitude,longitude
0,575150029610684424,广东深圳市宝安区,广东,广东深圳市,2,32.212549,119.92271
1,563888070781309192,辽宁营口市鲅鱼圈区芦屯镇,辽宁,辽宁营口市,1,40.285469,122.262267
2,563888070781309192,辽宁营口市鲅鱼圈区红海街道,辽宁,辽宁营口市,2,40.2489,122.139
3,563888070781309192,辽宁营口市鲅鱼圈区芦屯镇,辽宁,辽宁营口市,1,40.285469,122.262267
4,563888070781309192,辽宁营口市鲅鱼圈区芦屯镇,辽宁,辽宁营口市,1,40.285469,122.262267


In [84]:
Latitude

{'辽宁营口市鲅鱼圈区红海街道': 40.248937,
 '辽宁营口市鲅鱼圈区海星街道': 40.294837,
 '四川广元市利州区东坝街道': 32.436609999999995,
 '四川遂宁市蓬溪县县城内': 30.657490999999997,
 '江苏淮安市涟水县南集镇': 33.764573,
 '安徽六安市霍邱县河口镇': 32.141339,
 '河北唐山市乐亭县城区': 39.357229,
 '广西崇左市大新县龙门乡': 22.889127,
 '湖南邵阳市新宁县黄龙镇': 26.540533,
 '广西梧州市苍梧县沙头镇': 23.944079000000002,
 '江苏常州市金坛区朱林镇': 31.725281,
 '甘肃白银市平川区水泉乡': 36.901571999999994,
 '陕西咸阳市乾县新阳镇': 34.500976,
 '广西梧州市藤县金鸡镇': 23.232557999999997,
 '浙江杭州市余杭区临平街道': 30.448443,
 '江西九江市武宁县宋溪镇': 29.30785,
 '江西宜春市上高县锦江镇': 28.262545000000003,
 '河南新乡市红旗区小店镇': 35.313784999999996,
 '河南漯河市源汇区大刘镇': 33.527366,
 '广东清远市佛冈县龙山镇': 23.803615,
 '浙江宁波市象山县爵溪街道': 29.482938,
 '广东汕头市南澳县云澳镇': 23.413879,
 '河南驻马店市新蔡县佛阁寺镇': 32.69829,
 '河南南阳市卧龙区百里奚街道': 33.003743,
 '河南漯河市郾城区新店镇': 33.656702,
 '云南大理州巍山县大仓镇': 25.386131,
 '吉林长春市二道区(北环城路以南)': 43.958451000000004,
 '湖北襄阳市襄州区古驿镇': 32.294056,
 '安徽铜陵市枞阳县白湖乡': 30.974721999999996,
 '福建三明市沙县现代物流产业开发区': 26.446506,
 '甘肃平凉市静宁县城区': 35.434012,
 '江西吉安市井冈山市古城镇': 26.761241,
 '北京顺义区双丰街道': 40.172117,
 '四川南充市仪陇县永乐镇'

In [95]:
from math import radians, cos, sin, asin, sqrt
#计算两点间距离-m
def geodistance(lng1,lat1,lng2,lat2):
    lng1, lat1, lng2, lat2 = map(radians, [lng1, lat1, lng2, lat2])
    dlon=lng2-lng1
    dlat=lat2-lat1
    a=sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2 
    dis=2*asin(sqrt(a))*6371*1000
    return dis

def mygroup(group):
    addr_num = len(group.region.unique())
    count = group.shape[0]
    addr_more_region = group.region.value_counts().sort_values(ascending = False).index[0]
    addr_more_region_proportion = (group.region.value_counts().sort_values
                                   (ascending = False).iloc[0]/count)
    
    addr_more_region_pro = group.region_pro.value_counts().sort_values(ascending = False).index[0]
   
    addr_more_region_diji = group.region_diji.value_counts().sort_values(ascending = False).index[0]
    
    addr_more_region_city = group.region_city.value_counts().sort_values(ascending = False).index[0]
                                   
    addr_less_region = group.region.value_counts().sort_values().index[0]
    
    addr_more_Latitude = float(Latitude[addr_more_region])
    addr_more_longitude = float(longitude[addr_more_region])
    addr_less_Latitude = float(Latitude[addr_less_region])
    addr_less_longitude = float(longitude[addr_less_region])
    addr_dis = 0
    if addr_num > 1:
        addr_dis = geodistance(addr_more_longitude,addr_more_Latitude,
                               addr_less_longitude,addr_less_Latitude)
    
    indexs = {
        "addr_more_region_pro":addr_more_region_pro,
        "addr_more_region_diji":addr_more_region_diji,
        "addr_more_region_city":addr_more_region_city,
        
        "addr_more_region_proportion":addr_more_region_proportion,
        "addr_more_Latitude":addr_more_Latitude, 
        "addr_more_longitude":addr_more_longitude, 
        "addr_dis":addr_dis
    }
    return pd.Series(data=[indexs[c] for c in indexs], index=[c for c in indexs])

tmp1 = fulldata.groupby(by=['id']).apply(mygroup)
tmp1.insert(0,"id",tmp1.index)
tmp1


,id,addr_more_region_pro,addr_more_region_diji,addr_more_region_city,addr_more_region_proportion,addr_more_Latitude,addr_more_longitude,addr_dis
id,,,,,,,,
616266486771618056,616266486771618056,四川,1,四川资阳市,1.000000,30.796354,104.346430,0.000000e+00
616411495579783176,616411495579783176,河北,1,河北保定市,0.750000,23.310057,113.818621,2.797932e+06
616483294195749128,616483294195749128,山东,2,山东临沂市,1.000000,28.447361,119.849952,0.000000e+00
616547037910929672,616547037910929672,广东,2,广东中山市,1.000000,31.179870,120.365776,0.000000e+00
616550964970656008,616550964970656008,云南,2,云南昆明市,0.500000,29.050977,111.671999,8.394956e+05
616657485251481864,616657485251481864,江苏,2,江苏扬州市,0.750000,34.709004,113.721861,8.738135e+05
616758963400085768,616758963400085768,广西,2,广西北海市,1.000000,30.856143,120.744588,0.000000e+00
616772308744278024,616772308744278024,江苏,1,江苏苏州市,1.000000,36.192897,120.431146,0.000000e+00
616777971096621320,616777971096621320,广东,2,广东深圳市,1.000000,32.212549,119.922710,0.000000e+00


In [96]:
tmp1.to_csv("../dealinput/addr_recive_CountUnonehot.csv")